In [ ]:
# ***************code description -- Part 2**********************#
# 2 tasks are completed in this part:
# --1) Merge the 3 sections of info -- author information,paper informarion and Plumx information using doi and 1st author info

# --2) add in the research affiliation ranking info for the 1st author using the csv file 'CWUR.csv' 
#      The connection key is the affiliation name. Here hieracrhical matching method on names is conducted.

#  after run the code below, output file 'Final_author_list_withRK_0428.csv' is generated.

In [1]:
import pandas as pd
import re
import pandasql as ps

In [2]:
#######################################
#section 1: import data

###original file
org_df = pd.read_csv('SCORE.csv')

###paper level
###1.paper basic info
paper_info_df = pd.read_csv('Docu_DOI.csv')
paper_info_df_1 = paper_info_df.drop(['Unnamed: 0','Unnamed: 0.1','pii', 'pubmed_id','subtype', 'volume'
                                      , 'aggregationType', 'article_number','pageRange','source_id',
                                     'subtypeDescription','issueIdentifier','coverDisplayDate'] , axis='columns')
###2. plum metrics
plum_info_df = pd.read_csv('Docu_Plum.csv')
plum_info_df_1 = plum_info_df.drop(['Unnamed: 0'] , axis='columns')

###author level
author_info_df = pd.read_csv('Final_author_list.csv')
author_info_df_1 = author_info_df.drop(['Unnamed: 0'] , axis='columns')

In [3]:
### original file -- join key: DOI_CR
print(org_df.columns)
print(org_df.head(3))

Index(['title_CR', 'author_last_CR', 'author_first_CR', 'pub_year_CR',
       'DOI_CR', 'ISSN_CR', 'publication_CR', 'pub_short',
       'publication_standard', 'paper_id', 'source_WOS', 'author_full',
       'title_WOS', 'publication_WOS', 'language', 'keywords', 'keywords_plus',
       'abstract', 'author_address', 'reprint_address', 'author_email',
       'funding', 'publisher', 'publisher_address', 'ISSN_WOS', 'eISSN',
       'pub_year_WOS', 'volume', 'issue', 'DOI_WOS', 'WOS_category',
       'category', 'accession_number', 'pg', 'author_last_WOS',
       'author_first_WOS', 'published', 'pdf_filename', 'claim2_abstract',
       'coded_claim2', 'claim2_pg', 'claim2_start', 'claim2_box',
       'claim3a_concretehyp', 'coded_claim3a', 'claim3a_pg', 'claim3a_start',
       'claim3a_box', 'claim3b_testspec', 'coded_claim3b', 'claim3b_pg',
       'claim3b_start', 'claim3b_box', 'claim4_inftest', 'coded_claim4',
       'claim4_pg', 'claim4_start', 'claim4_box', 'sample_size',
       'co

In [4]:
###plum metrics --join key: doi
plum_info_df_1 = plum_info_df_1.add_prefix("paper_")
plum_info_df_1 = plum_info_df_1.rename(columns = {"paper_doi":"doi"})
print(plum_info_df_1.columns)
print(plum_info_df_1.head(3))

Index(['paper_ABSTRACT_VIEWS', 'paper_BOOKMARK_COUNT', 'paper_DOWNLOAD_COUNT',
       'paper_EXPORTS_SAVES', 'paper_FULL_TEXT_VIEWS',
       'paper_LINK_CLICK_COUNT', 'paper_LINK_OUTS', 'paper_READER_COUNT',
       'paper_capture', 'paper_citation', 'doi', 'paper_mention',
       'paper_socialMedia', 'paper_usage'],
      dtype='object')
   paper_ABSTRACT_VIEWS  paper_BOOKMARK_COUNT  paper_DOWNLOAD_COUNT  \
0                 370.0                   NaN                   NaN   
1                 157.0                   NaN                   NaN   
2                 219.0                   NaN                   NaN   

   paper_EXPORTS_SAVES  paper_FULL_TEXT_VIEWS  paper_LINK_CLICK_COUNT  \
0                 21.0                    NaN                     NaN   
1                  3.0                    NaN                     NaN   
2                  5.0                    NaN                     NaN   

   paper_LINK_OUTS  paper_READER_COUNT  paper_capture  paper_citation  \
0        

In [5]:
###author info --join key: author (represent authorID)
author_info_df_1 = author_info_df_1.rename(columns = {"citation_count":"author_citation_count", 'cited_by_count': "author_cited_by_count"
                                    , 'document_count': 'author_document_count', "h_index":"author_h_index"}) 
author_info_df_1 = author_info_df_1.drop('uniqueness' , axis='columns')
print(author_info_df_1.columns)
print(author_info_df_1.head(3))

# author_FN =  author_info_df_1['author_firstName']== 'Malia F.'
# print(author_info_df_1[author_FN])

Index(['Current_Affiliation_City', 'Current_Affiliation_Country',
       'Current_Affiliation_Domain', 'Current_Affiliation_Name', 'author',
       'author_firstName', 'author_lastName', 'author_citation_count',
       'author_cited_by_count', 'author_document_count', 'author_h_index',
       'history_affiliation_city', 'history_affiliation_country',
       'history_affiliation_name', 'history_affiliation_org_URL',
       'jounal_history_ISSN', 'jounal_history_name'],
      dtype='object')
  Current_Affiliation_City Current_Affiliation_Country  \
0                   Ithaca               United States   
1                 Montreal                      Canada   
2                Neuchatel                 Switzerland   

  Current_Affiliation_Domain                Current_Affiliation_Name  \
0        johnson.cornell.edu  Cornell SC Johnson College of Business   
1                  mcgill.ca                       McGill University   
2                   unine.ch                Cognitive Sc

In [6]:
#######################################
#section 2: process affiliation info in the doi table
paper_info_df_1 = paper_info_df_1.add_prefix("paper_")
paper_info_df_1 = paper_info_df_1.rename(columns = {"paper_doi":"doi", "paper_eid":"eid", 
                                                    "paper_FirstAuthor_ID":"FirstAuthor_ID",
                                                    "paper_affilname":"affilname", 
                                                    "paper_affiliation_city":"affiliation_city", "paper_affiliation_country":"affiliation_country",
                                                    "paper_afid":"afid"})
print(paper_info_df_1.columns)
print(paper_info_df_1.head(3))
print(len(paper_info_df_1))

Index(['eid', 'doi', 'paper_title', 'paper_creator', 'afid', 'affilname',
       'affiliation_city', 'affiliation_country', 'paper_author_count',
       'paper_author_names', 'paper_author_ids', 'paper_author_afids',
       'paper_coverDate', 'paper_publicationName', 'paper_issn', 'paper_eIssn',
       'paper_description', 'paper_authkeywords', 'paper_citedby_count',
       'paper_openaccess', 'paper_fund_acr', 'paper_fund_no',
       'paper_fund_sponsor', 'FirstAuthor_ID'],
      dtype='object')
                  eid                         doi  \
0  2-s2.0-85042850238  10.1016/j.jesp.2018.02.013   
1  2-s2.0-85046140774  10.1016/j.jesp.2018.04.001   
2  2-s2.0-85050101833  10.1016/j.jesp.2018.06.002   

                                         paper_title paper_creator  \
0  From belief to deceit: How expectancies about ...      Mason M.   
1  (Pre)occupations: A data-driven model of jobs ...     Imhoff R.   
2  CEOs imbue organizations with feelings, increa...       Tang S.   

    

In [7]:
###parse affiliation name for 1st author:
First_Author_affiliation_list = paper_info_df_1['affilname'].tolist()
# print(First_Author_affiliation_list)

afName_list = []
afName_final_list = []
for afName_parsed in First_Author_affiliation_list:
    print("------------------------------------------")
    afName_list = re.split(';', str(afName_parsed))
#     print(afName_list)
    print("1stAuthor id: ", afName_list[0].strip(' '))
    print("----------------------------------")
    afName_final_list.append(afName_list[0].strip(' '))
    
paper_info_df_1['FirstAuthor_AffiliationName'] = afName_final_list
print(paper_info_df_1.columns)
print(paper_info_df_1.head(3))

------------------------------------------
1stAuthor id:  Columbia Business School
----------------------------------
------------------------------------------
1stAuthor id:  Johannes Gutenberg Universität Mainz
----------------------------------
------------------------------------------
1stAuthor id:  The University of North Carolina at Chapel Hill
----------------------------------
------------------------------------------
1stAuthor id:  Ben-Gurion University of the Negev
----------------------------------
------------------------------------------
1stAuthor id:  University of St Andrews
----------------------------------
------------------------------------------
1stAuthor id:  The Ohio State University at Newark
----------------------------------
------------------------------------------
1stAuthor id:  Boston College
----------------------------------
------------------------------------------
1stAuthor id:  The Ohio State University
----------------------------------
---------

1stAuthor id:  Brigham Young University
----------------------------------
------------------------------------------
1stAuthor id:  University of California, San Diego
----------------------------------
------------------------------------------
1stAuthor id:  Loyola Marymount University
----------------------------------
------------------------------------------
1stAuthor id:  Stanford University
----------------------------------
------------------------------------------
1stAuthor id:  US Naval Academy
----------------------------------
------------------------------------------
1stAuthor id:  London School of Economics and Political Science
----------------------------------
------------------------------------------
1stAuthor id:  John F. Kennedy School of Government
----------------------------------
------------------------------------------
1stAuthor id:  High Point University
----------------------------------
------------------------------------------
1stAuthor id:  Nova Sc

------------------------------------------
1stAuthor id:  IE Business School
----------------------------------
------------------------------------------
1stAuthor id:  Indian School of Business
----------------------------------
------------------------------------------
1stAuthor id:  Columbia University in the City of New York
----------------------------------
------------------------------------------
1stAuthor id:  University of Central Florida
----------------------------------
------------------------------------------
1stAuthor id:  University of Toronto
----------------------------------
------------------------------------------
1stAuthor id:  ESMT Berlin
----------------------------------
------------------------------------------
1stAuthor id:  Kelley School of Business
----------------------------------
------------------------------------------
1stAuthor id:  Robert H. Smith School of Business
----------------------------------
------------------------------------------

------------------------------------------
1stAuthor id:  MRC Cognition and Brain Sciences Unit
----------------------------------
------------------------------------------
1stAuthor id:  University of Western Australia
----------------------------------
------------------------------------------
1stAuthor id:  University of Miami
----------------------------------
------------------------------------------
1stAuthor id:  University of Rochester
----------------------------------
------------------------------------------
1stAuthor id:  Stockholms Iäns Landsting
----------------------------------
------------------------------------------
1stAuthor id:  Aarhus Universitet
----------------------------------
------------------------------------------
1stAuthor id:  NHS Foundation Trust
----------------------------------
------------------------------------------
1stAuthor id:  Claremont Graduate University
----------------------------------
------------------------------------------
1st

In [8]:
#############################
###Section 4:parse affiliation city for 1st author:
First_Author_affiliationCity_list = paper_info_df_1['affiliation_city'].tolist()
# print(First_Author_affiliationCity_list)

afCityName_list = []
afCityName_final_list = []
for afCityName_parsed in First_Author_affiliationCity_list:
    print("------------------------------------------")
    afCityName_list = re.split(';', str(afCityName_parsed))
#     print(afCityName_list)
    print("1stAuthor city: ", afCityName_list[0].strip(' '))
    print("----------------------------------")
    afCityName_final_list.append(afCityName_list[0].strip(' '))
    
paper_info_df_1['FirstAuthor_AffiliationCity'] = afCityName_final_list
print(paper_info_df_1.columns)
print(paper_info_df_1.head(3))

------------------------------------------
1stAuthor city:  New York
----------------------------------
------------------------------------------
1stAuthor city:  Mainz
----------------------------------
------------------------------------------
1stAuthor city:  Chapel Hill
----------------------------------
------------------------------------------
1stAuthor city:  Beer Sheba
----------------------------------
------------------------------------------
1stAuthor city:  St Andrews
----------------------------------
------------------------------------------
1stAuthor city:  Newark
----------------------------------
------------------------------------------
1stAuthor city:  Chestnut Hill
----------------------------------
------------------------------------------
1stAuthor city:  Columbus
----------------------------------
------------------------------------------
1stAuthor city:  Athens
----------------------------------
------------------------------------------
1stAuthor city: 

1stAuthor city:  Coral Gables
----------------------------------
------------------------------------------
1stAuthor city:  Lowell
----------------------------------
------------------------------------------
1stAuthor city:  Storrs
----------------------------------
------------------------------------------
1stAuthor city:  Kolkata
----------------------------------
------------------------------------------
1stAuthor city:  Portland
----------------------------------
------------------------------------------
1stAuthor city:  Talence
----------------------------------
------------------------------------------
1stAuthor city:  Chestnut Hill
----------------------------------
------------------------------------------
1stAuthor city:  Chicago
----------------------------------
------------------------------------------
1stAuthor city:  Ann Arbor
----------------------------------
------------------------------------------
1stAuthor city:  Wufong
----------------------------------
--

------------------------------------------
1stAuthor city:  Madison
----------------------------------
------------------------------------------
1stAuthor city:  Austin
----------------------------------
------------------------------------------
1stAuthor city:  Princeton
----------------------------------
------------------------------------------
1stAuthor city:  Madison
----------------------------------
------------------------------------------
1stAuthor city:  Pittsburgh
----------------------------------
------------------------------------------
1stAuthor city:  Atlanta
----------------------------------
------------------------------------------
1stAuthor city:  Gothenburg
----------------------------------
------------------------------------------
1stAuthor city:  Hamburg
----------------------------------
------------------------------------------
1stAuthor city:  Durham
----------------------------------
------------------------------------------
1stAuthor city:  Modum
-

------------------------------------------
1stAuthor city:  Jyvaskyla
----------------------------------
------------------------------------------
1stAuthor city:  Edmonton
----------------------------------
------------------------------------------
1stAuthor city:  Berkeley
----------------------------------
------------------------------------------
1stAuthor city:  Munich
----------------------------------
------------------------------------------
1stAuthor city:  Bloomington
----------------------------------
------------------------------------------
1stAuthor city:  London
----------------------------------
------------------------------------------
1stAuthor city:  Maastricht
----------------------------------
------------------------------------------
1stAuthor city:  Iowa City
----------------------------------
------------------------------------------
1stAuthor city:  Davis
----------------------------------
------------------------------------------
1stAuthor city:  Colu

In [9]:
###parse affiliation country for 1st author:
First_Author_affiliationCountry_list = paper_info_df_1['affiliation_country'].tolist()
# print(First_Author_affiliationCountry_list)

afCountryName_list = []
afCountryName_final_list = []
for afCountryName_parsed in First_Author_affiliationCountry_list:
    print("------------------------------------------")
    afCountryName_list = re.split(';', str(afCountryName_parsed))
#     print(afCityName_list)
    print("1stAuthor country: ", afCountryName_list[0].strip(' '))
    print("----------------------------------")
    afCountryName_final_list.append(afCountryName_list[0].strip(' '))
    
paper_info_df_1['FirstAuthor_AffiliationCountry'] = afCountryName_final_list
print(paper_info_df_1.columns)
print(paper_info_df_1.head(3))

------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  Germany
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  Israel
----------------------------------
------------------------------------------
1stAuthor country:  United Kingdom
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
----

1stAuthor country:  United Kingdom
----------------------------------
------------------------------------------
1stAuthor country:  Switzerland
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  Netherlands
----------------------------------
------------------------------------------
1stAuthor country:  United Kingdom
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  Germany
----------------------------------
------------------------------------------


----------------------------------
------------------------------------------
1stAuthor country:  United Kingdom
----------------------------------
------------------------------------------
1stAuthor country:  Israel
----------------------------------
------------------------------------------
1stAuthor country:  Canada
----------------------------------
------------------------------------------
1stAuthor country:  Australia
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United Kingdom
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
--------

------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  Netherlands
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  Canada
----------------------------------
------------------------------------------
1stAuthor country:  China
----------------------------------
------------------------------------------
1stAuthor country:  Chile
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  United States
----------------------------------
------------------------------------------
1stAuthor country:  France
----------------------------------
------------------------

In [10]:
###parse affiliation ID for 1st author:
First_Author_affiliationID_list = paper_info_df_1['afid'].tolist()
# print(First_Author_affiliationID_list)

afIDName_list = []
afIDName_final_list = []
for afIDName_parsed in First_Author_affiliationID_list:
    print("------------------------------------------")
    afIDName_list = re.split(';', str(afIDName_parsed))
#     print(afCityName_list)
    print("1stAuthor country: ", afIDName_list[0].strip(' '))
    print("----------------------------------")
    afIDName_final_list.append(afIDName_list[0].strip(' '))
    
paper_info_df_1['FirstAuthor_AffiliationID'] = afIDName_final_list
print(paper_info_df_1.columns)
print(paper_info_df_1.head(3))

------------------------------------------
1stAuthor country:  60027552
----------------------------------
------------------------------------------
1stAuthor country:  60031216
----------------------------------
------------------------------------------
1stAuthor country:  60025111
----------------------------------
------------------------------------------
1stAuthor country:  60027161
----------------------------------
------------------------------------------
1stAuthor country:  60022132
----------------------------------
------------------------------------------
1stAuthor country:  60013177
----------------------------------
------------------------------------------
1stAuthor country:  60031117
----------------------------------
------------------------------------------
1stAuthor country:  60003500
----------------------------------
------------------------------------------
1stAuthor country:  60011132
----------------------------------
-------------------------------------

1stAuthor country:  60072562
----------------------------------
------------------------------------------
1stAuthor country:  60029278
----------------------------------
------------------------------------------
1stAuthor country:  60022132
----------------------------------
------------------------------------------
1stAuthor country:  60102127
----------------------------------
------------------------------------------
1stAuthor country:  60030807
----------------------------------
------------------------------------------
1stAuthor country:  60119490
----------------------------------
------------------------------------------
1stAuthor country:  60019816
----------------------------------
------------------------------------------
1stAuthor country:  60102124
----------------------------------
------------------------------------------
1stAuthor country:  60030162
----------------------------------
------------------------------------------
1stAuthor country:  60014171
--------

------------------------------------------
1stAuthor country:  60015186
----------------------------------
------------------------------------------
1stAuthor country:  60029311
----------------------------------
------------------------------------------
1stAuthor country:  60092858
----------------------------------
------------------------------------------
1stAuthor country:  100810804
----------------------------------
------------------------------------------
1stAuthor country:  60029747
----------------------------------
------------------------------------------
1stAuthor country:  60033389
----------------------------------
------------------------------------------
1stAuthor country:  60029747
----------------------------------
------------------------------------------
1stAuthor country:  60032179
----------------------------------
------------------------------------------
1stAuthor country:  60019816
----------------------------------
------------------------------------

----------------------------------
------------------------------------------
1stAuthor country:  60027509
----------------------------------
------------------------------------------
1stAuthor country:  60030612
----------------------------------
------------------------------------------
1stAuthor country:  60112612
----------------------------------
------------------------------------------
1stAuthor country:  60116134
----------------------------------
------------------------------------------
1stAuthor country:  60116392
----------------------------------
------------------------------------------
1stAuthor country:  60022195
----------------------------------
------------------------------------------
1stAuthor country:  60009982
----------------------------------
------------------------------------------
1stAuthor country:  60032775
----------------------------------
------------------------------------------
1stAuthor country:  nan
----------------------------------
-------

In [11]:
# ##check if still duplicated papaers under certain doi -- found 4 doi records -- drop incorrect ones
paper_info_cnt = paper_info_df_1.groupby(['doi'], sort=False).size().reset_index(name='doi_cnt')
# return 0 records means n duplicated author ID are founded
paper_info_df_withFreq = pd.merge(paper_info_df_1, paper_info_cnt, on='doi', how='inner')
doi_freq =  paper_info_df_withFreq['doi_cnt'] > 1
print(paper_info_df_withFreq[doi_freq])

                     eid                       doi  \
606   2-s2.0-84874872879       10.1509/jmr.10.0083   
607   2-s2.0-84870877576       10.1509/jmr.10.0083   
819   2-s2.0-84922324460  10.1177/0003122414558919   
820   2-s2.0-84922318631  10.1177/0003122414558919   
1105  2-s2.0-84922287099  10.1177/0003122414562600   
1106  2-s2.0-84922279342  10.1177/0003122414562600   
2449  2-s2.0-84870196607    10.1287/mnsc.1120.1542   
2450  2-s2.0-84870174224    10.1287/mnsc.1120.1542   

                                            paper_title paper_creator  \
606   It's not whether you win or lose, it's how you...       Yang X.   
607   It's not whether you win or lose, it's how you...       Yang X.   
819   Traditional, Modern, and Post-Secular Perspect...    O’Brien T.   
820   Traditional, Modern, and Post-Secular Perspect...    O’Brien T.   
1105  Capturing Culture: A New Method to Estimate Ex...  Polavieja J.   
1106  Capturing Culture: A New Method to Estimate Ex...  Polavieja J.   
24

In [12]:
print(len(paper_info_df_1))
# indexNames = dfObj[ (dfObj['Age'] >= 30) & (dfObj['Age'] <= 40) ].index
# dfObj.drop(indexNames , inplace=True)
indexNames  = paper_info_df_1[ (paper_info_df_1['eid'] == '2-s2.0-84870877576') | (paper_info_df_1['eid'] == '2-s2.0-84922318631')| (paper_info_df_1['eid'] == '2-s2.0-84922279342') | (paper_info_df_1['eid'] == '2-s2.0-84870196607')].index
paper_info_df_1.drop(indexNames , inplace=True)
print(len(paper_info_df_1))

2780
2776


In [13]:
# author_afName =  paper_info_df_1['FirstAuthor_AffiliationName']== 'nan'
# print(paper_info_df_1[author_afName])

In [14]:
#######################################
#section 4: merge table using sql logic
#org_df -- 'DOI_CR' -- 2804
#paper_info_df_1 --  'doi', 'FirstAuthor_ID' -- 2780
#plum_info_df_1 -- 'doi' -- 2801
#author_info_df_1 -- 'author' -- 2601
print(len(org_df))
print(len(paper_info_df_1))
print(len(plum_info_df_1))
print(len(author_info_df_1))

2804
2776
2801
2613


In [15]:
# print(author_duplicate_removed.columns)

# ---matching: author_info_df_1: 'Current_Affiliation_Name','Current_Affiliation_City', 'Current_Affiliation_Country'
# ---matching: paper_info_df_1: 'FirstAuthor_AffiliationName', 'FirstAuthor_AffiliationCity', 'FirstAuthor_AffiliationCountry'
q1 = """SELECT org.*, pl.*, pa.*, au.*,
               CASE WHEN pa.FirstAuthor_AffiliationName != 'nan' THEN pa.FirstAuthor_AffiliationName 
               ELSE au.Current_Affiliation_Name 
               END  AS FirstAuthor_Affiliation_Name,
               CASE WHEN pa.FirstAuthor_AffiliationCity != 'nan' THEN pa.FirstAuthor_AffiliationCity
               ELSE au.Current_Affiliation_City 
               END AS FirstAuthor_Affiliation_City,
               CASE WHEN pa.FirstAuthor_AffiliationCountry != 'nan' THEN pa.FirstAuthor_AffiliationCountry 
               ELSE au.Current_Affiliation_Country
               END AS FirstAuthor_Affiliation_Country
        FROM org_df org
         LEFT JOIN plum_info_df_1 pl on org.DOI_CR = pl.doi
         LEFT JOIN paper_info_df_1 pa on org.DOI_CR = pa.doi
         LEFT JOIN author_info_df_1 au on pa.FirstAuthor_ID = au.author"""

merged_df = ps.sqldf(q1, locals())
print(list(merged_df.columns))

['title_CR', 'author_last_CR', 'author_first_CR', 'pub_year_CR', 'DOI_CR', 'ISSN_CR', 'publication_CR', 'pub_short', 'publication_standard', 'paper_id', 'source_WOS', 'author_full', 'title_WOS', 'publication_WOS', 'language', 'keywords', 'keywords_plus', 'abstract', 'author_address', 'reprint_address', 'author_email', 'funding', 'publisher', 'publisher_address', 'ISSN_WOS', 'eISSN', 'pub_year_WOS', 'volume', 'issue', 'DOI_WOS', 'WOS_category', 'category', 'accession_number', 'pg', 'author_last_WOS', 'author_first_WOS', 'published', 'pdf_filename', 'claim2_abstract', 'coded_claim2', 'claim2_pg', 'claim2_start', 'claim2_box', 'claim3a_concretehyp', 'coded_claim3a', 'claim3a_pg', 'claim3a_start', 'claim3a_box', 'claim3b_testspec', 'coded_claim3b', 'claim3b_pg', 'claim3b_start', 'claim3b_box', 'claim4_inftest', 'coded_claim4', 'claim4_pg', 'claim4_start', 'claim4_box', 'sample_size', 'coded_sample_size', 'sample_size_pg', 'sample_size_start', 'sample_size_box', 'effect_size', 'coded_effect

In [16]:
merged_df = merged_df.drop(['Current_Affiliation_Name','Current_Affiliation_City', 'Current_Affiliation_Country',
                           'FirstAuthor_AffiliationName', 'FirstAuthor_AffiliationCity', 'FirstAuthor_AffiliationCountry',
                           'doi','doi'] , axis='columns')

In [17]:
print(list(merged_df.columns))

['title_CR', 'author_last_CR', 'author_first_CR', 'pub_year_CR', 'DOI_CR', 'ISSN_CR', 'publication_CR', 'pub_short', 'publication_standard', 'paper_id', 'source_WOS', 'author_full', 'title_WOS', 'publication_WOS', 'language', 'keywords', 'keywords_plus', 'abstract', 'author_address', 'reprint_address', 'author_email', 'funding', 'publisher', 'publisher_address', 'ISSN_WOS', 'eISSN', 'pub_year_WOS', 'volume', 'issue', 'DOI_WOS', 'WOS_category', 'category', 'accession_number', 'pg', 'author_last_WOS', 'author_first_WOS', 'published', 'pdf_filename', 'claim2_abstract', 'coded_claim2', 'claim2_pg', 'claim2_start', 'claim2_box', 'claim3a_concretehyp', 'coded_claim3a', 'claim3a_pg', 'claim3a_start', 'claim3a_box', 'claim3b_testspec', 'coded_claim3b', 'claim3b_pg', 'claim3b_start', 'claim3b_box', 'claim4_inftest', 'coded_claim4', 'claim4_pg', 'claim4_start', 'claim4_box', 'sample_size', 'coded_sample_size', 'sample_size_pg', 'sample_size_start', 'sample_size_box', 'effect_size', 'coded_effect

In [22]:
# print(merged_df.head(5))
print(author_info_df_1.columns)
# print(paper_info_df_1.columns)

Index(['Current_Affiliation_City', 'Current_Affiliation_Country',
       'Current_Affiliation_Domain', 'Current_Affiliation_Name', 'author',
       'author_firstName', 'author_lastName', 'author_citation_count',
       'author_cited_by_count', 'author_document_count', 'author_h_index',
       'history_affiliation_city', 'history_affiliation_country',
       'history_affiliation_name', 'history_affiliation_org_URL',
       'jounal_history_ISSN', 'jounal_history_name'],
      dtype='object')


In [19]:
print(len(merged_df))

2804


In [20]:
merged_df.to_csv('Final_data_list_0427.csv') 

In [ ]:
#############################
###Section 5: get academic affiliation ranking

In [23]:
###import ranking data
UNVRanking_df = pd.read_csv('affiliation_ranking_data/CWUR.csv')
# print("record count: ", len(UNVRanking_df))
# print(UNVRanking_df.head(3))
country_list = UNVRanking_df['Location'].tolist()
country_list = list(set(country_list))
print(country_list)
print("country count: ", len(country_list))

###import paper merged data
paper_author_df = pd.read_csv('Final_data_list_0427.csv')
paper_author_df['Record_Index'] = paper_author_df.index
affiliation_country_list = paper_author_df['FirstAuthor_Affiliation_Country'].tolist()
affiliation_country_list = list(set(affiliation_country_list))
print(affiliation_country_list)
print("country count: ", len(affiliation_country_list))

['Argentina', 'Nigeria', 'Democratic Republic of the Congo', 'Switzerland', 'Lebanon', 'Cyprus', 'Uganda', 'New Zealand', 'South Africa', 'Mexico', 'Tunisia', 'Morocco', 'Cameroon', 'Philippines', 'Croatia', 'Kuwait', 'Georgia', 'Sri Lanka', 'Spain', 'Belarus', 'North Macedonia', 'Latvia', 'Palestine', 'Singapore', 'Brazil', 'India', 'Australia', 'Bulgaria', 'Puerto Rico', 'Ghana', 'Taiwan', 'Qatar', 'Slovak Republic', 'Saudi Arabia', 'Zimbabwe', 'Ukraine', 'Malta', 'Costa Rica', 'Mozambique', 'Sweden', 'Iran', 'Canada', 'Israel', 'Egypt', 'Slovenia', 'Chile', 'Hungary', 'Senegal', 'Belgium', 'Tanzania', 'South Korea', 'Oman', 'Estonia', 'Malawi', 'Uruguay', 'Cuba', 'Lithuania', 'Armenia', 'China', 'Ireland', 'Iceland', 'Serbia', 'Greece', 'Colombia', 'Jamaica', 'Czech Republic', 'Poland', 'United Kingdom', 'Hong Kong', 'Indonesia', 'Kenya', 'Japan', 'Austria', 'Pakistan', 'Ethiopia', 'Denmark', 'Romania', 'Jordan', 'Italy', 'Zambia', 'Luxembourg', 'Norway', 'United States', 'Netherlan

In [24]:
paper_author_df = paper_author_df.drop(['Unnamed: 0'] , axis='columns')
print(list(paper_author_df.columns))
print(list(UNVRanking_df.columns))

['title_CR', 'author_last_CR', 'author_first_CR', 'pub_year_CR', 'DOI_CR', 'ISSN_CR', 'publication_CR', 'pub_short', 'publication_standard', 'paper_id', 'source_WOS', 'author_full', 'title_WOS', 'publication_WOS', 'language', 'keywords', 'keywords_plus', 'abstract', 'author_address', 'reprint_address', 'author_email', 'funding', 'publisher', 'publisher_address', 'ISSN_WOS', 'eISSN', 'pub_year_WOS', 'volume', 'issue', 'DOI_WOS', 'WOS_category', 'category', 'accession_number', 'pg', 'author_last_WOS', 'author_first_WOS', 'published', 'pdf_filename', 'claim2_abstract', 'coded_claim2', 'claim2_pg', 'claim2_start', 'claim2_box', 'claim3a_concretehyp', 'coded_claim3a', 'claim3a_pg', 'claim3a_start', 'claim3a_box', 'claim3b_testspec', 'coded_claim3b', 'claim3b_pg', 'claim3b_start', 'claim3b_box', 'claim4_inftest', 'coded_claim4', 'claim4_pg', 'claim4_start', 'claim4_box', 'sample_size', 'coded_sample_size', 'sample_size_pg', 'sample_size_start', 'sample_size_box', 'effect_size', 'coded_effect

In [25]:
# # split affiliation names
affiliation_name_l = paper_author_df['FirstAuthor_Affiliation_Name'].tolist()
afName_list = []
affiliation_name_list_final = []
for name in affiliation_name_l:
    afName_list = re.split(' ', str(name))
    afName_list = list(filter(lambda a: a != 'the', afName_list))
    afName_list = list(filter(lambda a: a != 'The', afName_list))
    afName_list = list(filter(lambda a: a != 'of', afName_list))
    afName_list = list(filter(lambda a: a != 'at', afName_list))
    afName_list = list(filter(lambda a: a != 'in', afName_list))
    afName_list = list(filter(lambda a: a != 'University', afName_list))
    afName_list = list(filter(lambda a: a != 'Institute', afName_list))    
    afName_list = list(filter(lambda a: a != 'Technology', afName_list))    
    afName_list = list(filter(lambda a: a != 'School', afName_list))
#     afName_list = list(filter(lambda a: a != 'College', afName_list))
    afName_list = list(filter(lambda a: a != 'California,', afName_list))
    afName_list = list(filter(lambda a: a != 'nan', afName_list))
    afName_list = list(filter(lambda a: a != 'and', afName_list))
    afName_list = list(filter(lambda a: a != '-', afName_list))
#     print(afName_list)
    affiliation_name_list_final.append(afName_list)
print(affiliation_name_list_final)   
paper_author_df['FirstAuthor_Affiliation_Name_List'] = affiliation_name_list_final    

# split institution names
Institution_name_l = UNVRanking_df['Institution'].tolist()
InstName_list = []
Institution_name_list_final = []
for name in Institution_name_l:
    InstName_list = re.split(' ', str(name))
    InstName_list = list(filter(lambda a: a != 'the', InstName_list))
    InstName_list = list(filter(lambda a: a != 'The', InstName_list))
    InstName_list = list(filter(lambda a: a != 'of', InstName_list))
    InstName_list = list(filter(lambda a: a != 'at', InstName_list))
    InstName_list = list(filter(lambda a: a != 'in', InstName_list))
    InstName_list = list(filter(lambda a: a != 'University', InstName_list))
    InstName_list = list(filter(lambda a: a != 'Institute', InstName_list))
    InstName_list =list(filter(lambda a: a != 'Technology', InstName_list)) 
    InstName_list = list(filter(lambda a: a != 'School', InstName_list))
#     InstName_list = list(filter(lambda a: a != 'College', InstName_list))
    InstName_list = list(filter(lambda a: a != 'California,', InstName_list))
    InstName_list = list(filter(lambda a: a != 'nan', InstName_list))
    InstName_list = list(filter(lambda a: a != 'and', InstName_list))
    InstName_list = list(filter(lambda a: a != '-', InstName_list))
#     print(InstName_list)
    Institution_name_list_final.append(InstName_list)
print(Institution_name_list_final)  
UNVRanking_df['Institution_List'] = Institution_name_list_final

[['Columbia', 'Business'], ['Johannes', 'Gutenberg', 'Universität', 'Mainz'], ['North', 'Carolina', 'Chapel', 'Hill'], ['Ben-Gurion', 'Negev'], ['St', 'Andrews'], ['Ohio', 'State', 'Newark'], ['Boston', 'College'], ['Ohio', 'State'], ['Ohio'], ['UC', 'Merced'], ['Doctorate-Granting', 'Institution', 'City', 'New', 'York'], ['Universiteit', 'Gent'], ['Texas', 'System'], ['Université', 'Clermont', 'Auvergne'], ['Universidad', 'Autónoma', 'de', 'Madrid'], ['Denver'], ['Berkeley'], ['IDC'], ['Northwestern'], ['Nevada,', 'Las', 'Vegas'], ['Chicago', 'Booth', 'Business'], ['Ben-Gurion', 'Negev'], ['Ludwig-Maximilians-Universität', 'München'], ['Leiden'], ['Hong', 'Kong', 'Polytechnic'], ['Loyola', 'New', 'Orleans'], ['Universitat', 'Salzburg'], ['Nanjing'], ['Alaska', 'Southeast'], ['Brandeis'], ['St.', 'Louis'], ['Universität', 'Bielefeld'], ['Universität', 'Hamburg'], ['Stephen', 'M.', 'Ross', 'Business'], ['New', 'York'], ['Technische', 'Universiteit', 'Eindhoven'], ['Fonds', 'de', 'la', '

In [26]:
print(UNVRanking_df.columns)
UNVRanking_tuples = [tuple(x) for x in UNVRanking_df.to_numpy()]
# print(UNVRanking_dftuples)

Index(['World Rank', 'Institution', 'Location', 'National Rank',
       'Quality of Education', 'Alumni Employment', 'Quality of Faculty',
       'Research Performance', 'Score', 'Institution_List'],
      dtype='object')


In [27]:
paper_AFsubset = paper_author_df[['Record_Index', 'FirstAuthor_Affiliation_Name', 'FirstAuthor_Affiliation_Name_List', 'FirstAuthor_Affiliation_Country']]
paper_AFtuples = [tuple(x) for x in paper_AFsubset.to_numpy()]
print(paper_AFtuples)

Rank_subset = UNVRanking_df[['World Rank', 'Institution','Institution_List', 'Location']]
Rank_subtuples = [tuple(x) for x in Rank_subset.to_numpy()]
# print(Rank_subtuples)

[(0, 'Columbia Business School', ['Columbia', 'Business'], 'United States'), (1, 'Johannes Gutenberg Universität Mainz', ['Johannes', 'Gutenberg', 'Universität', 'Mainz'], 'Germany'), (2, 'The University of North Carolina at Chapel Hill', ['North', 'Carolina', 'Chapel', 'Hill'], 'United States'), (3, 'Ben-Gurion University of the Negev', ['Ben-Gurion', 'Negev'], 'Israel'), (4, 'University of St Andrews', ['St', 'Andrews'], 'United Kingdom'), (5, 'The Ohio State University at Newark', ['Ohio', 'State', 'Newark'], 'United States'), (6, 'Boston College', ['Boston', 'College'], 'United States'), (7, 'The Ohio State University', ['Ohio', 'State'], 'United States'), (8, 'Ohio University', ['Ohio'], 'United States'), (9, 'UC Merced', ['UC', 'Merced'], 'United States'), (10, 'The Doctorate-Granting Institution of the City University of New York', ['Doctorate-Granting', 'Institution', 'City', 'New', 'York'], 'United States'), (11, 'Universiteit Gent', ['Universiteit', 'Gent'], 'Belgium'), (12, 

In [28]:
list0 = []
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []

# 'World Rank', 'Institution', 'Location', 'National Rank',
#        'Quality of Education', 'Alumni Employment', 'Quality of Faculty',
#        'Research Performance', 'Score', 'Institution_List'

for World_rank, Institution_Name, Institution_country, National_Rank, Education_Quality, Alumni_Employment, Faculty_Quality, Research_Performance, Ranking_Score, Institution_Key in UNVRanking_tuples:
    for Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country in paper_AFtuples:
        if Institution_country == Affiliation_Country:
            Institution_Name = Institution_Name.strip(' ')
            Affiliation_Name = Affiliation_Name.strip(' ')
            if Institution_Name == Affiliation_Name:
#                 print("rank info: ----")
#                 print((World_rank, Institution_Name, Institution_Key, Institution_country))
#                 print("affiliation info: ----")
#                 print((Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country))
#                 print("-----------------------------------------------------------------------")
                list0_tuple = (Index, World_rank, National_Rank, Education_Quality, Alumni_Employment, Faculty_Quality, Research_Performance, Institution_Name, Institution_country)
                list0.append(list0_tuple)
                
            elif len(set(Institution_Key) & set(Affiliation_Name_Key)) ==6:
#                 print("rank info: ----")
#                 print((World_rank, Institution_Name, Institution_Key, Institution_country))
#                 print("affiliation info: ----")
#                 print((Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country))
#                 print("-----------------------------------------------------------------------")
                list1_tuple = (World_rank, Institution_Name, Index)
                list1.append(list1_tuple)
            elif len(set(Institution_Key) & set(Affiliation_Name_Key)) ==5:
                print("rank info: ----")
                print((World_rank, Institution_Name, Institution_Key, Institution_country))
                print("affiliation info: ----")
                print((Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country))
                print("-----------------------------------------------------------------------")
                list2_tuple = (Index, World_rank, National_Rank, Education_Quality, Alumni_Employment, Faculty_Quality, Research_Performance, Institution_Name, Institution_country)
                list2.append(list2_tuple)
            elif len(set(Institution_Key) & set(Affiliation_Name_Key)) ==4:
                if 'publique' not in Affiliation_Name_Key and 'publique' in Institution_Key:
                    continue
                elif Institution_Name == 'Queens College, City University of New York' and Affiliation_Name == 'City College of New York':
                    continue
                elif Institution_Name == 'City College of New York' and Affiliation_Name != 'City College of New York':
                    continue   
                else:
#                     print("rank info: ----")
#                     print((World_rank, Institution_Name, Institution_Key, Institution_country))
#                     print("affiliation info: ----")
#                     print((Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country))
#                     print("-----------------------------------------------------------------------")
                    list3_tuple = (Index, World_rank, National_Rank, Education_Quality, Alumni_Employment, Faculty_Quality, Research_Performance, Institution_Name, Institution_country)
                    list3.append(list3_tuple)
            elif len(set(Institution_Key) & set(Affiliation_Name_Key)) ==3:
                if 'State' in Affiliation_Name_Key and 'State' not in Institution_Key:
                    continue
                elif 'City' in Affiliation_Name_Key and 'City' not in Institution_Key:
                    continue
                elif 'Bank' in Affiliation_Name_Key and 'Bank' not in Institution_Key:
                    continue
                elif 'California' not in Affiliation_Name_Key and 'California' in Institution_Key:
                    continue
                elif 'Science' not in Affiliation_Name_Key and 'Science' in Institution_Key:
                    continue
                elif 'Science' in Affiliation_Name_Key and 'Science' not in Institution_Key:
                    continue
                elif 'Polytechnic'  in Affiliation_Name_Key and 'Polytechnic' not in Institution_Key:
                    continue
                elif 'Polytechnic'  not in Affiliation_Name_Key and 'Polytechnic' in Institution_Key:
                    continue
                elif 'Huazhong' in Affiliation_Name_Key and 'Huazhong' not in Institution_Key:
                    continue
                elif 'Duke' not in Affiliation_Name_Key and 'Duke'  in Institution_Key:
                    continue
                elif 'A&T' not in Affiliation_Name_Key and 'A&T'  in Institution_Key:
                    continue
                elif 'East' not in Affiliation_Name_Key and 'East'  in Institution_Key:
                    continue
                elif 'Baptist' not in Affiliation_Name_Key and 'Baptist'  in Institution_Key:
                    continue
                elif 'Baptist' in Affiliation_Name_Key and 'Baptist'  not in Institution_Key:
                    continue
                elif 'Chinese' in Affiliation_Name_Key and 'Chinese' not in Institution_Key:
                    continue
                elif 'Chinese' not in Affiliation_Name_Key and 'Chinese' in Institution_Key:
                    continue
                elif 'Hawaii' in Affiliation_Name_Key and 'Hawaii' not in Institution_Key:
                    continue
                elif 'Brownsville' in Affiliation_Name_Key and 'Brownsville' not in Institution_Key:
                    continue
                elif 'Mary' in Affiliation_Name_Key and 'Mary' not in Institution_Key:
                    continue
                elif 'Dakota' in Affiliation_Name_Key and 'Dakota' not in Institution_Key:
                    continue
                elif 'Chapel' in Affiliation_Name_Key and 'Chapel' not in Institution_Key:
                    continue
                elif 'Chapel' not in Affiliation_Name_Key and 'Chapel' in Institution_Key:
                    continue
                elif 'City' not in Affiliation_Name_Key and 'City' in Institution_Key:
                    continue
                elif 'State' not in Affiliation_Name_Key and 'State' in Institution_Key:
                    continue
                elif 'Oklahoma' not in Affiliation_Name_Key and 'Oklahoma' in Institution_Key:
                    continue
                elif 'Medical' not in Affiliation_Name_Key and 'Medical' in Institution_Key:
                    continue
                elif 'Ramapo' in Affiliation_Name_Key and 'Ramapo' not in Institution_Key:
                    continue
                elif 'Air' in Affiliation_Name_Key and 'Air' not in Institution_Key:
                    continue
                elif 'Military' not in Affiliation_Name_Key and 'Military' in Institution_Key:
                    continue
                elif 'Air' not in Affiliation_Name_Key and 'Air' in Institution_Key:
                    continue
                elif 'Wales' in Affiliation_Name_Key and 'Wales' not in Institution_Key:
                    continue
                elif 'Greensboro' not in Affiliation_Name_Key and 'Greensboro' in Institution_Key:
                    continue
                elif 'Ocean' not in Affiliation_Name_Key and 'Ocean' in Institution_Key:
                    continue
                elif 'Northridge' not in Affiliation_Name_Key and 'Northridge' in Institution_Key:
                    continue
                elif 'Mining' not in Affiliation_Name_Key and 'Mining' in Institution_Key:
                    continue
                elif 'Human' in Affiliation_Name_Key and 'Human' not in Institution_Key:
                    continue
                elif 'National' in Affiliation_Name_Key and 'National' not in Institution_Key:
                    continue
                elif 'Center' in Affiliation_Name_Key and 'Center' not in Institution_Key:
                    continue
                elif 'Mining' not in Affiliation_Name_Key and 'Mining' in Institution_Key:
                    continue
                elif 'Wilmington' not in Affiliation_Name_Key and 'Wilmington'  in Institution_Key:
                    continue
                elif 'Education' not in Affiliation_Name_Key and 'Education' in Institution_Key:
                    continue
                elif 'Advanced' not in Affiliation_Name_Key and 'Advanced' in Institution_Key:
                    continue
                elif 'Calcutta' not in Affiliation_Name_Key and 'Calcutta'  in Institution_Key:
                    continue
                elif 'Angeles' not in Affiliation_Name_Key and 'Angeles' in Institution_Key:
                    continue
                elif 'Tennessee' not in Affiliation_Name_Key and 'Tennessee' in Institution_Key:
                    continue
                elif 'supérieur' not in Affiliation_Name_Key and 'supérieur'  in Institution_Key:
                    continue
                elif 'Changhua' not in Affiliation_Name_Key and 'Changhua' in Institution_Key:
                    continue
                elif 'Sacramento' not in Affiliation_Name_Key and 'Sacramento' in Institution_Key:
                    continue
                elif 'supérieur' not in Affiliation_Name_Key and 'supérieur'  in Institution_Key:
                    continue
                elif 'supérieure' not in Affiliation_Name_Key and 'supérieure' in Institution_Key:
                    continue
                elif 'appliquées' not in Affiliation_Name_Key and 'appliquées' in Institution_Key:
                    continue
                elif 'Southwestern' not in Affiliation_Name_Key and 'Southwestern'  in Institution_Key:
                    continue
                elif 'supérieure' not in Affiliation_Name_Key and 'supérieure' in Institution_Key:
                    continue
                elif 'appliquées' not in Affiliation_Name_Key and 'appliquées' in Institution_Key:
                    continue
                elif 'London' not in Affiliation_Name_Key and 'London' in Institution_Key:
                    continue
                elif 'Ahmedabad' not in Affiliation_Name_Key and 'Ahmedabad' in Institution_Key:
                    continue
                elif 'ParisTech' not in Affiliation_Name_Key and 'ParisTech' in Institution_Key:
                    continue
                elif 'Nebraska' not in Affiliation_Name_Key and 'Nebraska' in Institution_Key:
                    continue
                elif 'SUNY' not in Affiliation_Name_Key and 'SUNY' in Institution_Key:
                    continue
                elif 'Lille' not in Affiliation_Name_Key and 'Lille' in Institution_Key:
                    continue
                elif 'Charlotte' not in Affiliation_Name_Key and 'Charlotte' in Institution_Key:
                    continue
                elif 'Nanjing' not in Affiliation_Name_Key and 'Nanjing' in Institution_Key:
                    continue
                elif 'Mexico' not in Affiliation_Name_Key and 'Mexico' in Institution_Key:
                    continue
                elif 'Advanced' in Affiliation_Name_Key and 'Advanced' not in Institution_Key:
                    continue
                elif 'Francisco' not in Affiliation_Name_Key and 'Francisco' in Institution_Key:
                    continue
                elif 'centrale' not in Affiliation_Name_Key and 'centrale' in Institution_Key:
                    continue
                elif 'Fullerton' in Affiliation_Name_Key and 'Fullerton' not in Institution_Key:
                    continue
                elif 'Fresno' not in Affiliation_Name_Key and 'Fresno' in Institution_Key:
                    continue
                elif 'Chung' not in Affiliation_Name_Key and 'Chung' in Institution_Key:
                    continue
                elif 'Naval' in Affiliation_Name_Key and 'Naval' not in Institution_Key:
                    continue
                elif 'Kaohsiung' not in Affiliation_Name_Key and 'Kaohsiung' in Institution_Key:
                    continue
                elif 'Chung' not in Affiliation_Name_Key and 'Chung' in Institution_Key:
                    continue
                elif 'Texas' not in Affiliation_Name_Key and 'Texas' in Institution_Key:
                    continue
                elif 'VA' in Affiliation_Name_Key and 'VA' not in Institution_Key:
                    continue
                elif 'Chongqing' not in Affiliation_Name_Key and 'Chongqing' in Institution_Key:
                    continue
                elif 'Yunlin' not in Affiliation_Name_Key and 'Yunlin' in Institution_Key:
                    continue
                elif 'Northern' in Affiliation_Name_Key and 'Northern' not in Institution_Key:
                    continue
                elif 'Pingtung' not in Affiliation_Name_Key and 'Pingtung' in Institution_Key:
                    continue
                elif 'Dongbei' in Affiliation_Name_Key and 'Dongbei' not in Institution_Key:
                    continue
                elif 'Dongbei' not in Affiliation_Name_Key and 'Dongbei' in Institution_Key:
                    continue
                elif 'centrale' not in Affiliation_Name_Key and 'centrale' in Institution_Key:
                    continue
                elif 'West' not in Affiliation_Name_Key and 'West' in Institution_Key:
                    continue
                elif 'Central' not in Affiliation_Name_Key and 'Central' in Institution_Key:
                    continue
                elif 'Columbia' in Affiliation_Name_Key and 'Columbia' not in Institution_Key:
                    continue
                elif 'Fullerton' not in Affiliation_Name_Key and 'Fullerton' in Institution_Key:
                    continue
                elif 'Beach' not in Affiliation_Name_Key and 'Beach' in Institution_Key:
                    continue
                elif 'Social' not in Affiliation_Name_Key and 'Social' in Institution_Key:
                    continue
                elif 'Naval' not in Affiliation_Name_Key and 'Naval' in Institution_Key:
                    continue
                elif 'Northern' not in Affiliation_Name_Key and 'Northern' in Institution_Key:
                    continue
                elif 'Trinity' not in Affiliation_Name_Key and 'Trinity' in Institution_Key:
                    continue
                elif 'Trinity'  in Affiliation_Name_Key and 'Trinity' not in Institution_Key:
                    continue
                elif 'Jose' not in Affiliation_Name_Key and 'Jose' in Institution_Key:
                    continue
                elif 'Diego'  in Affiliation_Name_Key and 'Diego' not in Institution_Key:
                    continue
                elif 'East'  not in Affiliation_Name_Key and 'East' in Institution_Key:
                    continue
                elif 'South'  not in Affiliation_Name_Key and 'South' in Institution_Key:
                    continue
                elif 'West'  not in Affiliation_Name_Key and 'West' in Institution_Key:
                    continue
                elif 'North'  not in Affiliation_Name_Key and 'North' in Institution_Key:
                    continue
                elif Institution_Name == 'University of San Francisco' and Affiliation_Name == 'University of California, San Francisco':
                    continue
                elif Institution_Name == 'University of San Diego' and Affiliation_Name == 'University of California, San Diego':
                    continue
                elif Institution_Name == 'University of California, San Francisco' and Affiliation_Name == 'San Francisco VA Health Care System':
                    continue
                elif Institution_Name == 'Queens College, City University of New York' and Affiliation_Name == 'City College of New York':
                    continue
                elif Institution_Name == 'City College of New York' and Affiliation_Name != 'City College of New York':
                    continue 
                elif Institution_Name == 'National Taiwan University' and Affiliation_Name != 'National Taiwan University':
                    continue
                elif 'Kaohsiung'  in Affiliation_Name_Key and 'Kaohsiung' not in Institution_Key:
                    continue
                elif 'Pratique'  in Affiliation_Name_Key and 'Pratique' not in Institution_Key:
                    continue
                elif 'Pratique'  not in Affiliation_Name_Key and 'Pratique' in Institution_Key:
                    continue
                else:
#                     print("rank info: ----")
#                     print((World_rank, Institution_Name, Institution_Key, Institution_country))
#                     print("affiliation info: ----")
#                     print((Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country))
#                     print("-----------------------------------------------------------------------")
                    list4_tuple = (Index, World_rank, National_Rank, Education_Quality, Alumni_Employment, Faculty_Quality, Research_Performance, Institution_Name, Institution_country)
                    list4.append(list4_tuple)
            elif len(set(Institution_Key) & set(Affiliation_Name_Key)) == 2:
                if 'State' in Affiliation_Name_Key and 'State' not in Institution_Key:
                    continue
                elif 'City' in Affiliation_Name_Key and 'City' not in Institution_Key:
                    continue
                elif 'Bank' in Affiliation_Name_Key and 'Bank' not in Institution_Key:
                    continue
                elif 'California' not in Affiliation_Name_Key and 'California' in Institution_Key:
                    continue
                elif 'Science' not in Affiliation_Name_Key and 'Science' in Institution_Key:
                    continue
                elif 'Science' in Affiliation_Name_Key and 'Science' not in Institution_Key:
                    continue
                elif 'Polytechnic'  in Affiliation_Name_Key and 'Polytechnic' not in Institution_Key:
                    continue
                elif 'Polytechnic'  not in Affiliation_Name_Key and 'Polytechnic' in Institution_Key:
                    continue
                elif 'Huazhong' in Affiliation_Name_Key and 'Huazhong' not in Institution_Key:
                    continue
                elif 'Duke' not in Affiliation_Name_Key and 'Duke'  in Institution_Key:
                    continue
                elif 'A&T' not in Affiliation_Name_Key and 'A&T'  in Institution_Key:
                    continue
                elif 'East' not in Affiliation_Name_Key and 'East'  in Institution_Key:
                    continue
                elif 'Baptist' not in Affiliation_Name_Key and 'Baptist'  in Institution_Key:
                    continue
                elif 'Baptist' in Affiliation_Name_Key and 'Baptist'  not in Institution_Key:
                    continue
                elif 'Chinese' in Affiliation_Name_Key and 'Chinese' not in Institution_Key:
                    continue
                elif 'Chinese' not in Affiliation_Name_Key and 'Chinese' in Institution_Key:
                    continue
                elif 'Hawaii' in Affiliation_Name_Key and 'Hawaii' not in Institution_Key:
                    continue
                elif 'Brownsville' in Affiliation_Name_Key and 'Brownsville' not in Institution_Key:
                    continue
                elif 'Mary' in Affiliation_Name_Key and 'Mary' not in Institution_Key:
                    continue
                elif 'Dakota' in Affiliation_Name_Key and 'Dakota' not in Institution_Key:
                    continue
                elif 'Chapel' in Affiliation_Name_Key and 'Chapel' not in Institution_Key:
                    continue
                elif 'Chapel' not in Affiliation_Name_Key and 'Chapel' in Institution_Key:
                    continue
                elif 'City' not in Affiliation_Name_Key and 'City' in Institution_Key:
                    continue
                elif 'State' not in Affiliation_Name_Key and 'State' in Institution_Key:
                    continue
                elif 'Oklahoma' not in Affiliation_Name_Key and 'Oklahoma' in Institution_Key:
                    continue
                elif 'Medical' not in Affiliation_Name_Key and 'Medical' in Institution_Key:
                    continue
                elif 'Ramapo' in Affiliation_Name_Key and 'Ramapo' not in Institution_Key:
                    continue
                elif 'Air' in Affiliation_Name_Key and 'Air' not in Institution_Key:
                    continue
                elif 'Military' not in Affiliation_Name_Key and 'Military' in Institution_Key:
                    continue
                elif 'Air' not in Affiliation_Name_Key and 'Air' in Institution_Key:
                    continue
                elif 'Wales' in Affiliation_Name_Key and 'Wales' not in Institution_Key:
                    continue
                elif 'Greensboro' not in Affiliation_Name_Key and 'Greensboro' in Institution_Key:
                    continue
                elif 'Ocean' not in Affiliation_Name_Key and 'Ocean' in Institution_Key:
                    continue
                elif 'Northridge' not in Affiliation_Name_Key and 'Northridge' in Institution_Key:
                    continue
                elif 'Mining' not in Affiliation_Name_Key and 'Mining' in Institution_Key:
                    continue
                elif 'Human' in Affiliation_Name_Key and 'Human' not in Institution_Key:
                    continue
                elif 'National' in Affiliation_Name_Key and 'National' not in Institution_Key:
                    continue
                elif 'Center' in Affiliation_Name_Key and 'Center' not in Institution_Key:
                    continue
                elif 'Mining' not in Affiliation_Name_Key and 'Mining' in Institution_Key:
                    continue
                elif 'Wilmington' not in Affiliation_Name_Key and 'Wilmington'  in Institution_Key:
                    continue
                elif 'Education' not in Affiliation_Name_Key and 'Education' in Institution_Key:
                    continue
                elif 'Advanced' not in Affiliation_Name_Key and 'Advanced' in Institution_Key:
                    continue
                elif 'Calcutta' not in Affiliation_Name_Key and 'Calcutta'  in Institution_Key:
                    continue
                elif 'Angeles' not in Affiliation_Name_Key and 'Angeles' in Institution_Key:
                    continue
                elif 'Tennessee' not in Affiliation_Name_Key and 'Tennessee' in Institution_Key:
                    continue
                elif 'supérieur' not in Affiliation_Name_Key and 'supérieur'  in Institution_Key:
                    continue
                elif 'Changhua' not in Affiliation_Name_Key and 'Changhua' in Institution_Key:
                    continue
                elif 'Sacramento' not in Affiliation_Name_Key and 'Sacramento' in Institution_Key:
                    continue
                elif 'supérieur' not in Affiliation_Name_Key and 'supérieur'  in Institution_Key:
                    continue
                elif 'supérieure' not in Affiliation_Name_Key and 'supérieure' in Institution_Key:
                    continue
                elif 'appliquées' not in Affiliation_Name_Key and 'appliquées' in Institution_Key:
                    continue
                elif 'Southwestern' not in Affiliation_Name_Key and 'Southwestern'  in Institution_Key:
                    continue
                elif 'supérieure' not in Affiliation_Name_Key and 'supérieure' in Institution_Key:
                    continue
                elif 'appliquées' not in Affiliation_Name_Key and 'appliquées' in Institution_Key:
                    continue
                elif 'London' not in Affiliation_Name_Key and 'London' in Institution_Key:
                    continue
                elif 'Ahmedabad' not in Affiliation_Name_Key and 'Ahmedabad' in Institution_Key:
                    continue
                elif 'ParisTech' not in Affiliation_Name_Key and 'ParisTech' in Institution_Key:
                    continue
                elif 'Nebraska' not in Affiliation_Name_Key and 'Nebraska' in Institution_Key:
                    continue
                elif 'SUNY' not in Affiliation_Name_Key and 'SUNY' in Institution_Key:
                    continue
                elif 'Lille' not in Affiliation_Name_Key and 'Lille' in Institution_Key:
                    continue
                elif 'Charlotte' not in Affiliation_Name_Key and 'Charlotte' in Institution_Key:
                    continue
                elif 'Nanjing' not in Affiliation_Name_Key and 'Nanjing' in Institution_Key:
                    continue
                elif 'Mexico' not in Affiliation_Name_Key and 'Mexico' in Institution_Key:
                    continue
                elif 'Advanced' in Affiliation_Name_Key and 'Advanced' not in Institution_Key:
                    continue
                elif 'Francisco' not in Affiliation_Name_Key and 'Francisco' in Institution_Key:
                    continue
                elif 'centrale' not in Affiliation_Name_Key and 'centrale' in Institution_Key:
                    continue
                elif 'Fullerton' in Affiliation_Name_Key and 'Fullerton' not in Institution_Key:
                    continue
                elif 'Fresno' not in Affiliation_Name_Key and 'Fresno' in Institution_Key:
                    continue
                elif 'Chung' not in Affiliation_Name_Key and 'Chung' in Institution_Key:
                    continue
                elif 'Naval' in Affiliation_Name_Key and 'Naval' not in Institution_Key:
                    continue
                elif 'Naval' not in Affiliation_Name_Key and 'Naval' in Institution_Key:
                    continue
                elif 'Kaohsiung' not in Affiliation_Name_Key and 'Kaohsiung' in Institution_Key:
                    continue
                elif 'Kaohsiung'  in Affiliation_Name_Key and 'Kaohsiung' not in Institution_Key:
                    continue
                elif 'Chung' not in Affiliation_Name_Key and 'Chung' in Institution_Key:
                    continue
                elif 'Texas' not in Affiliation_Name_Key and 'Texas' in Institution_Key:
                    continue
                elif 'VA' in Affiliation_Name_Key and 'VA' not in Institution_Key:
                    continue
                elif 'Chongqing' not in Affiliation_Name_Key and 'Chongqing' in Institution_Key:
                    continue
                elif 'Yunlin' not in Affiliation_Name_Key and 'Yunlin' in Institution_Key:
                    continue
                elif 'Northern' in Affiliation_Name_Key and 'Northern' not in Institution_Key:
                    continue
                elif 'Northern' not in Affiliation_Name_Key and 'Northern' in Institution_Key:
                    continue
                elif 'Pingtung' not in Affiliation_Name_Key and 'Pingtung' in Institution_Key:
                    continue
                elif 'Dongbei' in Affiliation_Name_Key and 'Dongbei' not in Institution_Key:
                    continue
                elif 'Dongbei' not in Affiliation_Name_Key and 'Dongbei' in Institution_Key:
                    continue
                elif 'centrale' not in Affiliation_Name_Key and 'centrale' in Institution_Key:
                    continue
                elif 'West' not in Affiliation_Name_Key and 'West' in Institution_Key:
                    continue
                elif 'Central' not in Affiliation_Name_Key and 'Central' in Institution_Key:
                    continue
                elif 'Columbia' in Affiliation_Name_Key and 'Columbia' not in Institution_Key:
                    continue
                elif 'Fullerton' not in Affiliation_Name_Key and 'Fullerton' in Institution_Key:
                    continue
                elif 'Beach' not in Affiliation_Name_Key and 'Beach' in Institution_Key:
                    continue
                elif 'Social' not in Affiliation_Name_Key and 'Social' in Institution_Key:
                    continue
                elif 'Trinity' not in Affiliation_Name_Key and 'Trinity' in Institution_Key:
                    continue
                elif 'Trinity'  in Affiliation_Name_Key and 'Trinity' not in Institution_Key:
                    continue
                elif 'Jose' not in Affiliation_Name_Key and 'Jose' in Institution_Key:
                    continue
                elif 'Diego'  in Affiliation_Name_Key and 'Diego' not in Institution_Key:
                    continue
                elif 'East'  not in Affiliation_Name_Key and 'East' in Institution_Key:
                    continue
                elif 'South'  not in Affiliation_Name_Key and 'South' in Institution_Key:
                    continue
                elif 'West'  not in Affiliation_Name_Key and 'West' in Institution_Key:
                    continue
                elif 'North'  not in Affiliation_Name_Key and 'North' in Institution_Key:
                    continue
                elif 'Pratique'  in Affiliation_Name_Key and 'Pratique' not in Institution_Key:
                    continue
                elif 'Pratique'  not in Affiliation_Name_Key and 'Pratique' in Institution_Key:
                    continue
                elif Institution_Name == 'University of San Francisco' and Affiliation_Name == 'University of California, San Francisco':
                    continue
                elif Institution_Name == 'University of San Diego' and Affiliation_Name == 'University of California, San Diego':
                    continue
                elif Institution_Name == 'University of California, San Francisco' and Affiliation_Name == 'San Francisco VA Health Care System':
                    continue
                elif Institution_Name == 'Queens College, City University of New York' and Affiliation_Name == 'City College of New York':
                    continue
                elif Institution_Name == 'City College of New York' and Affiliation_Name != 'City College of New York':
                    continue
                elif Institution_Name == 'Texas Tech University Health Sciences Center' and Affiliation_Name != 'Texas Tech University Health Sciences Center':
                    continue 
                elif Institution_Name == 'National Taiwan University' and Affiliation_Name != 'National Taiwan University':
                    continue
                else:
                    print("rank info: ----")
                    print((World_rank, Institution_Name, Institution_Key, Institution_country))
                    print("affiliation info: ----")
                    print((Index, Affiliation_Name, Affiliation_Name_Key, Affiliation_Country))
                    print("-----------------------------------------------------------------------")
                    list5_tuple = (Index, World_rank, National_Rank, Education_Quality, Alumni_Employment, Faculty_Quality, Research_Performance, Institution_Name, Institution_country)
                    list5.append(list5_tuple)
            else:
                continue
        else:
            continue

rank info: ----
(18, 'Johns Hopkins University', ['Johns', 'Hopkins'], 'United States')
affiliation info: ----
(1873, 'Johns Hopkins Bloomberg School of Public Health', ['Johns', 'Hopkins', 'Bloomberg', 'Public', 'Health'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(31, 'University of Texas at Austin', ['Texas', 'Austin'], 'United States')
affiliation info: ----
(508, 'The University of Texas at Austin', ['Texas', 'Austin'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(31, 'University of Texas at Austin', ['Texas', 'Austin'], 'United States')
affiliation info: ----
(652, 'The University of Texas at Austin', ['Texas', 'Austin'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(31, 'University of Texas at Austin', ['Texas', 'Austin'], 'United States')
affiliation info: ----
(666, 'The University of Texas at Au

rank info: ----
(98, 'Erasmus University Rotterdam', ['Erasmus', 'Rotterdam'], 'Netherlands')
affiliation info: ----
(38, 'Erasmus Universiteit Rotterdam', ['Erasmus', 'Universiteit', 'Rotterdam'], 'Netherlands')
-----------------------------------------------------------------------
rank info: ----
(98, 'Erasmus University Rotterdam', ['Erasmus', 'Rotterdam'], 'Netherlands')
affiliation info: ----
(52, 'Rotterdam School of Management, Erasmus University', ['Rotterdam', 'Management,', 'Erasmus'], 'Netherlands')
-----------------------------------------------------------------------
rank info: ----
(98, 'Erasmus University Rotterdam', ['Erasmus', 'Rotterdam'], 'Netherlands')
affiliation info: ----
(336, 'Rotterdam School of Management, Erasmus University', ['Rotterdam', 'Management,', 'Erasmus'], 'Netherlands')
-----------------------------------------------------------------------
rank info: ----
(98, 'Erasmus University Rotterdam', ['Erasmus', 'Rotterdam'], 'Netherlands')
affiliation 

rank info: ----
(264, 'Indiana University – Purdue University Indianapolis', ['Indiana', '–', 'Purdue', 'Indianapolis'], 'United States')
affiliation info: ----
(1852, 'Indiana University-Purdue University Indianapolis', ['Indiana', 'University-Purdue', 'Indianapolis'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(273, 'University of Tennessee, Knoxville', ['Tennessee,', 'Knoxville'], 'United States')
affiliation info: ----
(194, 'The University of Tennessee, Knoxville', ['Tennessee,', 'Knoxville'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(273, 'University of Tennessee, Knoxville', ['Tennessee,', 'Knoxville'], 'United States')
affiliation info: ----
(1677, 'The University of Tennessee, Knoxville', ['Tennessee,', 'Knoxville'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(276, 'University of New Mexico',

rank info: ----
(483, 'Oklahoma State University', ['Oklahoma', 'State'], 'United States')
affiliation info: ----
(65, 'Oklahoma State University - Stillwater', ['Oklahoma', 'State', 'Stillwater'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(483, 'Oklahoma State University', ['Oklahoma', 'State'], 'United States')
affiliation info: ----
(321, 'Oklahoma State University System', ['Oklahoma', 'State', 'System'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(483, 'Oklahoma State University', ['Oklahoma', 'State'], 'United States')
affiliation info: ----
(325, 'Oklahoma State University - Stillwater', ['Oklahoma', 'State', 'Stillwater'], 'United States')
-----------------------------------------------------------------------
rank info: ----
(483, 'Oklahoma State University', ['Oklahoma', 'State'], 'United States')
affiliation info: ----
(1102, 'Oklahoma State University

In [29]:
print(len(list0))
print(len(list(set(list0))))


986
986


In [30]:
print(len(list1))
print(len(list(set(list1))))

0
0


In [31]:
print(len(list2))
print(len(list(set(list2))))

1
1


In [32]:
print(len(list3))
print(len(list(set(list3))))

22
22


In [33]:
print(len(list4))
print(len(list(set(list4))))

27
27


In [34]:
print(len(list5))
print(len(list(set(list5))))

134
134


In [35]:
986 + 0 +1 +22 +27 +134

1170

In [36]:
finalList = list0 + list1 + list2 + list3 + list4 + list5
print(finalList[0])

(599, 1, 1, '2', '1', '1', '1', 'Harvard University', 'United States')


In [37]:
ranking_df_with_Index = pd.DataFrame(finalList, columns=['Index', 'CWUR_World_rank', 'CWUR_National_Rank', 'CWUR_Education_Quality', 'CWUR_Alumni_Employment', 'CWUR_Faculty_Quality', 'CWUR_Research_Performance', 'CWUR_Institution_Name', 'CWUR_Institution_country'])
print(ranking_df_with_Index.head(10))

   Index  CWUR_World_rank  CWUR_National_Rank CWUR_Education_Quality  \
0    599                1                   1                      2   
1    632                1                   1                      2   
2    707                1                   1                      2   
3    713                1                   1                      2   
4    824                1                   1                      2   
5    917                1                   1                      2   
6    971                1                   1                      2   
7    975                1                   1                      2   
8   1127                1                   1                      2   
9   1210                1                   1                      2   

  CWUR_Alumni_Employment CWUR_Faculty_Quality CWUR_Research_Performance  \
0                      1                    1                         1   
1                      1                    1            

In [41]:
# paper_author_df.columns.get_loc("eIssn")
print(list(paper_author_df.columns))
paper_author_df_1 = paper_author_df.drop(['paper_issn', 'paper_eIssn'] , axis='columns')
paper_author_df_1 = paper_author_df.drop('FirstAuthor_Affiliation_Name_List' , axis='columns')

['title_CR', 'author_last_CR', 'author_first_CR', 'pub_year_CR', 'DOI_CR', 'ISSN_CR', 'publication_CR', 'pub_short', 'publication_standard', 'paper_id', 'source_WOS', 'author_full', 'title_WOS', 'publication_WOS', 'language', 'keywords', 'keywords_plus', 'abstract', 'author_address', 'reprint_address', 'author_email', 'funding', 'publisher', 'publisher_address', 'ISSN_WOS', 'eISSN', 'pub_year_WOS', 'volume', 'issue', 'DOI_WOS', 'WOS_category', 'category', 'accession_number', 'pg', 'author_last_WOS', 'author_first_WOS', 'published', 'pdf_filename', 'claim2_abstract', 'coded_claim2', 'claim2_pg', 'claim2_start', 'claim2_box', 'claim3a_concretehyp', 'coded_claim3a', 'claim3a_pg', 'claim3a_start', 'claim3a_box', 'claim3b_testspec', 'coded_claim3b', 'claim3b_pg', 'claim3b_start', 'claim3b_box', 'claim4_inftest', 'coded_claim4', 'claim4_pg', 'claim4_start', 'claim4_box', 'sample_size', 'coded_sample_size', 'sample_size_pg', 'sample_size_start', 'sample_size_box', 'effect_size', 'coded_effect

In [42]:
ranking_df_with_Index = ranking_df_with_Index.rename(columns = {"Index":"Record_Index"}) 

In [43]:
index_record_cnt = ranking_df_with_Index.groupby(['Record_Index'], sort=False).size().reset_index(name='index_cnt')
rk_info_df_withFreq = pd.merge(ranking_df_with_Index, index_record_cnt, on='Record_Index', how='inner')
# index_freq =  rk_info_df_withFreq['index_cnt'] == 2
dup_data  = rk_info_df_withFreq[rk_info_df_withFreq['index_cnt'] >= 2]

print(dup_data[['Record_Index', 'CWUR_World_rank','CWUR_Institution_Name']])

Empty DataFrame
Columns: [Record_Index, CWUR_World_rank, CWUR_Institution_Name]
Index: []


In [44]:
merge_df_1 = pd.merge(paper_author_df_1, ranking_df_with_Index, on='Record_Index', how='left')

In [45]:
print(len(merge_df_1))
print(list(merge_df_1.columns))

2804
['title_CR', 'author_last_CR', 'author_first_CR', 'pub_year_CR', 'DOI_CR', 'ISSN_CR', 'publication_CR', 'pub_short', 'publication_standard', 'paper_id', 'source_WOS', 'author_full', 'title_WOS', 'publication_WOS', 'language', 'keywords', 'keywords_plus', 'abstract', 'author_address', 'reprint_address', 'author_email', 'funding', 'publisher', 'publisher_address', 'ISSN_WOS', 'eISSN', 'pub_year_WOS', 'volume', 'issue', 'DOI_WOS', 'WOS_category', 'category', 'accession_number', 'pg', 'author_last_WOS', 'author_first_WOS', 'published', 'pdf_filename', 'claim2_abstract', 'coded_claim2', 'claim2_pg', 'claim2_start', 'claim2_box', 'claim3a_concretehyp', 'coded_claim3a', 'claim3a_pg', 'claim3a_start', 'claim3a_box', 'claim3b_testspec', 'coded_claim3b', 'claim3b_pg', 'claim3b_start', 'claim3b_box', 'claim4_inftest', 'coded_claim4', 'claim4_pg', 'claim4_start', 'claim4_box', 'sample_size', 'coded_sample_size', 'sample_size_pg', 'sample_size_start', 'sample_size_box', 'effect_size', 'coded_e

In [47]:
# save csv for data with acedemic rankings
merge_df_1.to_csv('Final_author_list_withRK_0428.csv') 